thanks to https://www.kaggle.com/h053473666/siim-cov19-yolov5-train/data?select=image

Making YoloV5 format

In [ ]:
import numpy as np, pandas as pd
from glob import glob
import os, shutil
import matplotlib as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns

In [ ]:
fold = 0

In [ ]:
train_df = pd.read_csv("../input/siim-covid19-detection/train_image_level.csv")
train_df.head()

In [ ]:
df = train_df
gkf = GroupKFold(n_splits = 5)
df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(df, groups = df.StudyInstanceUID.tolist())):
    df.loc[val_idx, 'fold'] = fold


In [ ]:
df

In [ ]:
train_df = df

In [ ]:
train_df['image_path'] = f'../input/second/image/' + train_df.id + '.png'
train_df

In [ ]:
train_files = []
val_files = []
val_files += list(train_df[train_df.fold==fold].image_path.unique())
train_files += list(train_df[train_df.fold!=fold].image_path.unique())
len(train_files), len(val_files)

In [ ]:
count = 0
for _,_,files in os.walk('../input/thirdyolotxt'):
    for file in files:
        count += 1
print(count)

In [ ]:
os.makedirs('/kaggle/working/siim-cov19/labels/train', exist_ok = True)
os.makedirs('/kaggle/working/siim-cov19/labels/val', exist_ok = True)
os.makedirs('/kaggle/working/siim-cov19/images/train', exist_ok = True)
os.makedirs('/kaggle/working/siim-cov19/images/val', exist_ok = True)
label_dir = '../input/thirdyolotxt'


for file in tqdm(val_files):
    shutil.copy(file, '/kaggle/working/siim-cov19/images/val')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename + '.txt'),'/kaggle/working/siim-cov19/labels/val')
    
for file in tqdm(train_files):
    shutil.copy(file, '/kaggle/working/siim-cov19/images/train')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename + '.txt'), '/kaggle/working/siim-cov19/labels/train')



In [ ]:
from os import listdir
from os.path import isfile, join
import yaml

cwd ='/kaggle/working/'

with open(join(cwd, 'train.txt'), 'w') as f:
    for path in glob('./siim-cov19/images/train/*'):
        f.write(path+'\n')
        
with open(join(cwd, 'val.txt'), 'w') as f:
    for path in glob('./siim-cov19/images/val/*'):
        f.write(path+'\n')

In [ ]:
data = dict(
    train = join(cwd, 'train.txt'),
    val = join(cwd, 'val.txt'),
    nc = 1,
    names = ['0. opacity']
)

with open(join(cwd, 'siim-cov19.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style = False)

In [ ]:
f = open(join(cwd, 'siim-cov19.yaml'), 'r')
print('\nyaml:')
print(f.read())

In [ ]:
!git clone https://github.com/ultralytics/yolov5


In [ ]:
%cd yolov5
%pip install -r requirements.txt

In [ ]:
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
# shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
# os.chdir('/kaggle/working/yolov5')

In [ ]:
# !WANDB_MODE="dryrun" python train.py --img 512 --batch 24 --epochs 1 --data /kaggle/working/siim-cov19.yaml \
# --weights yolov5s.pt --project=/kaggle/working/results --nosave --exist-ok --cache

In [ ]:
os.makedirs('/kaggle/working/results', exist_ok = True)

In [ ]:

!WANDB_MODE="dryrun" python train.py --img 512 --batch 24 --epochs 30 --data /kaggle/working/siim-cov19.yaml --weights yolov5l.pt \
--project=/kaggle/working/results --nosave --exist-ok --cache

# !python train.py --img 512 --batch 16 --epochs 30 --data /kaggle/working/siim-cov19.yaml --weights yolov5l.pt \
# --project=/kaggle/working/results --exist-ok --cache